# Task 0.a: Data Volume Estimation

Based on the task description here are general functions which can calculate the estimates for us, and give us a sense of the data volume we are dealing with.

---

1. Assume you are pulling 4 metrics from the Fitbit (heart_rate, steps,
distance,spo2) at a 1 second resolution for 1 year**.

- a. How many data points is this for n=1? n=1,000? n=10,000?

- b. How many data points is this for 1 year? 2 years? 5 years?

In [2]:
def num_datapoints(num_metrics, resolution_in_seconds, num_years, num_participants):
    """
    This function calculate the number of datapoints which we'll have to collect.

    Parameters:
    num_metrics: Number of metrics being measured.
    resolution_in_seconds: Frequency of data collection in seconds.
    num_years: Number of years the study runs.
    num_participants: Number of participants in the study.

    Returns:
    int: Total number of data points collected.
    """
    
    year_in_seconds = 60 * 60 * 24 * 365
    total_seconds = num_years * year_in_seconds
    datapoints_per_metric = total_seconds / resolution_in_seconds
    total_datapoints = datapoints_per_metric * num_metrics * num_participants
    return int(total_datapoints)

In [3]:
# a)
print(f"Estimated datapoints for n = 1: {num_datapoints(4, 1, 1, 1)}")
print(f"Estimated datapoints for n = 1000: {num_datapoints(4, 1, 1, 1000)}")
print(f"Estimated datapoints for n = 10000: {num_datapoints(4, 1, 1, 10000)}")

Estimated datapoints for n = 1: 126144000
Estimated datapoints for n = 1000: 126144000000
Estimated datapoints for n = 10000: 1261440000000


In [4]:
# b) checking for 1 participant
print(f"Estimated datapoints for 1 year: {num_datapoints(4, 1, 1, 1)}")
print(f"Estimated datapoints for 2 years: {num_datapoints(4, 1, 2, 1)}")
print(f"Estimated datapoints for 5 years: {num_datapoints(4, 1, 5, 1)}")

Estimated datapoints for 1 year: 126144000
Estimated datapoints for 2 years: 252288000
Estimated datapoints for 5 years: 630720000


---

2. To store 1 data point, how many bytes of data should this take?
    
- a. For n=1,000 / 2 years / 3 metrics at 1 second resolutions, how many bytes of
    data will this take if uncompressed (e.g. stored in native PostgreSQL)?
    
- b. A compressed time-series database (e.g. TimescaleDB) is commonly quoted to
    be able to compress data by >90% (a, b). Assuming a conservative compression
    rate of 80%, how much data is this?
    
- - i. How can time-series databases compress data by such a large %? In
        what cases would time-series data not be compressed by very much?
        Would health data (e.g. heart rate, sleep, physical activity data, etc)
        extracted from a Fitbit be a good or bad example of data that can be
        compressed significantly? Why or why not?

In [5]:
def num_bytes(num_datapoints, byte_size):
    """
    This function calculates the total number of bytes required to store the data.

    Parameters:
    num_datapoints: Total number of data points collected.
    byte_size: Normally in postgreSQL there are int2, int4, int8 types, so it's based on the metric what size will we need.  

    Returns:
    int: Total number of bytes required.
    """
    
    return num_datapoints * byte_size

def data_compression(bytes, compression_rate):
    """
    This function calculates the size of the data after compression.

    Parameters:
    bytes: Total number of bytes before compression.
    compression_rate: Compression rate as a percentage (e.g., 0.5 for 50% compression).

    Returns:
    int: Total number of bytes after compression.
    """
    
    return int(bytes * (1 - compression_rate))

In [6]:
# a) let's assume that we want the metric to use int4
total_bytes = num_bytes(num_datapoints(3, 1, 2, 1000), 4)
print(f"Estimated bytes in PostgreSQL: {total_bytes}")
print(f"Estimated data requirements in GB: {total_bytes / (1024 ** 3):.4f}")

Estimated bytes in PostgreSQL: 756864000000
Estimated data requirements in GB: 704.8845


In [7]:
# b) if we're using timescaleDB and data is able to compress by a rate of 80%
compressed_bytes = data_compression(total_bytes, 0.8)
print(f"Estimated bytes after compression: {compressed_bytes}")
print(f"Estimated data requirements in GB after compression: {compressed_bytes / (1024 ** 3):.4f}")

Estimated bytes after compression: 151372799999
Estimated data requirements in GB after compression: 140.9769


How are time series db's able to do this?

Answer: There are a few reasons why any time-series db does this,
- Delta encoding: When we have data this is sequential, we can store the difference between the current and previous value instead of the full value. This is especially useful for metrics that change slowly over time, like heart rate or steps. An example for this would be like this
- - For example, if the heart rate is 70 bpm at time t1 and 72 bpm at time t2, instead of storing both values, we can store the delta (2 bpm) and the timestamp.
- - Similar things can be done with time stamps, where as an example if we save the timestamp in seconds, we can store the difference between the current timestamp and the previous one, rather than the full timestamp value.
- Delta to Delta encoding: This is a further optimization where we store the difference between the current delta and the previous delta, which can lead to even smaller storage requirements. 
- - Incase of let's say time, if the resolution is 30 second, we will always get the same delta of 30 second always, so instead of storing values like t, t+30, t+60, t+90, our values would be t, 30, 0, 0 because the difference between deltas is 0.  
- Hypertables: For TimescaleDB specifically, it uses hypertables to partition data into smaller chunks based on time intervals which can later be compressed indpendently, also allowing efficient queries.
- Dictionary encoding: This is a technique where we store unique values in a dictionary and replace the actual values with references to the dictionary. This is useful when there are many repeated values, an example of this would be categorical data, like if there some sensor recording the person's movement status, it could be idle, moving, running etc, these categories definitely take more space than something like int2, so we can store the unique values in a dictionary and replace the actual values with references to the dictionary.
- Run-length encoding: If something is repeated a lot of times, for example if a sensor notes down the sleep status of someone every 1 hour, the sensor will give us one of two values which will be sleeping or not sleeping. If someone sleeps for 8 hours that value will be repeated 8 times, so instead of storing sleeping, sleeping, sleeping, ... 8 times we can store something like a tuple (sleeping, 8) which will bascially tell us how many times this value is repeated.

There are many other techniques as well, but these are common and some of these are found in TimescaleDB as well.

However there are some cases where time-series data can be very hard to compress, examples of such data could be data with no apparent patterns, like if we have a sensor that records the temperature every second, and the temperature changes randomly every second, then it will be very hard to compress this data. But in the case of health data, like heart rate, sleep, physical activity data, etc, these are usually periodic and have patterns, so they can be compressed significantly.

---

3. For questions 1 and 2, we assumed 3 metrics at a 1 second resolution. In reality,
depending on the metric, the resolution can vary considerably. Look into the Fitbit Web
API explorer. What may be some useful metrics to run a physical activity affecting sleep
study?

- a. List the metrics and their highest frequency (e.g. 1 sec, 1 min, 5 min, 15 min, etc)
- b. What is the actual volume of data that is produced for n=1,000 for a study
duration of 1 year?
- c. What is the compressed volume of data for above? (e.g. assume 80%
compression)

Answer: 

a) There are a few metrics that can be useful for such a study, and those are the following:

- Activity (highest frequency: 1 minute)
- Active Zone Minutes (highest frequency: 1 minute)
- Sleep (highest frequency: 1 day)
- Heart Rate (highest frequency: 1 second)
- Heart Rate Variability (highest frequency: 5 minutes)

Other can also be used (SpO2, Temperature, Breathing Rate, etc) but these are probably the most useful ones in a physical activity affecting sleep study.<br/>Note: I'm assuming we consider the intraday data too, even though it is mentioned that it might require special permissions. 

In [8]:
# b) Consider that we use the above mentioned 5 metrics this is how our calculation would look like
SECONDS_IN_MINUTE = 60
SECONDS_IN_HOUR = 60 * SECONDS_IN_MINUTE
SECONDS_IN_DAY = 24 * SECONDS_IN_HOUR

activity_datapoints = num_datapoints(num_metrics=1, num_participants=1000, num_years=1, resolution_in_seconds=SECONDS_IN_MINUTE)
azm_datapoints = num_datapoints(num_metrics=1, num_participants=1000, num_years=1, resolution_in_seconds=SECONDS_IN_MINUTE)
sleep_datapoints = num_datapoints(num_metrics=1, num_participants=1000, num_years=1, resolution_in_seconds=SECONDS_IN_DAY)
heart_rate_datapoints = num_datapoints(num_metrics=1, num_participants=1000, num_years=1, resolution_in_seconds=1)
hrv_datapoints = num_datapoints(num_metrics=1, num_participants=1000, num_years=1, resolution_in_seconds=5*SECONDS_IN_MINUTE)

total_datapoints = activity_datapoints + azm_datapoints + sleep_datapoints + heart_rate_datapoints + hrv_datapoints

# assuming we use int4 for all metrics
total_bytes = num_bytes(total_datapoints, 4)
print(f"Total estimated bytes for all metrics in PostgreSQL: {total_bytes}")
print(f"Total estimated data requirements in GB for all metrics: {total_bytes / (1024 ** 3):.4f}")

Total estimated bytes for all metrics in PostgreSQL: 130770740000
Total estimated data requirements in GB for all metrics: 121.7897


In [9]:
# c) if we assume a compression rate of 80% for all metrics we will get the following result
compressed_bytes = data_compression(total_bytes, 0.8)
print(f"Total estimated bytes after compression for all metrics: {compressed_bytes}")
print(f"Total estimated data requirements in GB after compression for all metrics: {compressed_bytes / (1024 ** 3):.4f}")

Total estimated bytes after compression for all metrics: 26154147999
Total estimated data requirements in GB after compression for all metrics: 24.3579


---

4. When retrieving time-series data from a database, it may be too expensive an operation
to access all the data at the finest resolution.
- a. How would you solve this issue? What would you do to make queries “less
expensive”? (Hint: this may come at an added data storage cost)

Answer:

a) To solve this problem, we can keep multiple resolutions of the same data, by downsampling data. DBs like timescaleDB support real time aggregation, often mentioned as TimescaleDB continuous aggregates which are materialized views that are automatically updated as new data is inserted. This allows us to query the data at different resolutions without having to access all the data at the finest resolution. But this comes at an added storage cost, as we will be storing multiple copies of the same data at different resolutions.

---

5. The above assumes a single server / machine. While modern day servers are able to
store and process high volumes of data at high capacities, vertical scaling has its limits.
- a. First, what would be a feasible limit for vertically scaling this? List CPU, memory,
and hard disk limitations on one machine.
- b. If we were to scale this service horizontally, what considerations would be
important? How would you resolve queries across multiple machines?
Remember, this is not being built on the cloud, so you would physically have
multiple machines next to each other.

Answer:

a) The feasible limits for vertically scaling a single machine can vary based on the hardware, but generally, we can consider the following:
- For CPU modern workstation CPUs are 64-128 cores, per socket.
- For memory, even though the theoretical limit of 64 bit architecture is 16 exabytes, practical limits are much lower. For example, modern servers can support up to 2TB of RAM, where usually the motherboard is the bottleneck.
- For hard disk, the theoretical limit is effectively unlimited, but practical limits are usually in the range of few hundered TBs to few PBs.

But all of this is based on budget constraints, as the more modern, top of the line hardware we use, the more expensive it gets. This isn't really the case when we scale the service horizontally, even though that can also be expensive, but we can usually use commodity hardware to scale horizontally.

b) When scaling horizontally, some important considerations would be:
- When partitioning the data, we will need to decide how to distribute the data across multiple machines. This can be done based on time intervals, as that is a common way to partition time-series data.
- We will need to have a coordinator node that can handle queries between multiple machines, like let's say we are storing the data in 12 machines each for 1 month of data of the year. If he need to query the data for April to June, we will need to implement the coordinator node in a way that it will query the machines responsible for those months and aggregate the results and return those to us.
- However we don't really need to implement this ourselves, because time series databases do often horizontal scaling support. For example, timescaleDB has native multi node support where it creates distributed hypertables that can be queried across multiple nodes. This allows us to scale horizontally without having to implement the coordination logic ourselves. But we will still have to manage the following:
- - Network overhead can be a concern but since machines are next to each other, the latency should be low.
- - Data replication is also very important in our case, we will need to ensure that there are copies of the data across multiple machines to ensure fault tolerance, this sort of replication is normally implemented at node level when working with cloud databases, but we will have to implement this ourselves if we are using a self hosted database. Luckily, timescaleDB has built in replication support, so we can use that to ensure that our data is replicated across multiple machines.
